In [1]:
import sys
import os
import fastText

from validation import compute_f1

from keras.models import load_model
from keras.callbacks import ModelCheckpoint, Callback

import models
import utils

/usr/lib/python3/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/usr/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: numpy.dtype size changed, may indicate binary incompatibility. Expected 96, got 88
  return f(*args, **kwds)


In [2]:
#%env CUDA_DEVICE_ORDER=PCI_BUS_ID
#%env CUDA_VISIBLE_DEVICES=0

In [3]:
trainSentences = utils.get_sentences_germeval('../../Resources/GermEVAL/NER-de-train.tsv', level2=True)
devSentences = utils.get_sentences_germeval('../../Resources/GermEVAL/NER-de-dev.tsv', level2=True)
testSentences = utils.get_sentences_germeval('../../Resources/GermEVAL/NER-de-test.tsv', level2=True)

print(len(trainSentences))
print(len(devSentences))
print(len(testSentences))


24000
2200
5100


In [4]:
print(testSentences[3])

[['Die', 'O'], ['These', 'O'], [',', 'O'], ['Schlatter', 'O'], ['sei', 'O'], ['Antisemit', 'O'], ['gewesen', 'O'], [',', 'O'], ['wurde', 'O'], ['seither', 'O'], ['in', 'O'], ['der', 'O'], ['theologischen', 'O'], ['Fachliteratur', 'O'], ['nicht', 'O'], ['mehr', 'O'], ['vertreten', 'O'], ['.', 'O']]


In [5]:
labelSet = set()
characters= set()
models.max_sequence_length = 0

for dataset in [trainSentences, devSentences, testSentences]:
    for sentence in dataset:
        for word, label in sentence:
            for char in word:
                characters.add(char)
            labelSet.add(label)
        if len(sentence) > models.max_sequence_length:
            models.max_sequence_length = len(sentence)

In [6]:
print(len(labelSet))
print(models.max_sequence_length)

18
56


In [7]:
# :: Create a mapping for the labels ::
models.label2Idx = {"PADDING_TOKEN":0}
for label in labelSet:
    models.label2Idx[label] = len(models.label2Idx)

In [8]:
print(models.label2Idx)

{'I-PER': 1, 'I-OTH': 2, 'O': 3, 'B-PERpart': 4, 'B-ORG': 6, 'B-OTHpart': 5, 'B-OTH': 7, 'B-LOCderiv': 8, 'B-PERderiv': 9, 'B-LOC': 10, 'PADDING_TOKEN': 0, 'B-ORGderiv': 11, 'I-ORG': 12, 'I-LOCderiv': 13, 'B-LOCpart': 14, 'B-ORGpart': 15, 'B-PER': 16, 'I-LOC': 17, 'B-OTHderiv': 18}


In [9]:
# :: Hard coded case lookup ::
models.case2Idx = {'PADDING_TOKEN':0, 'numeric': 1, 'allLower':2, 'allUpper':3, 'initialUpper':4, 'other':5, 'mainly_numeric':6, 'contains_digit': 7}

In [10]:
print(models.case2Idx)

{'numeric': 1, 'allLower': 2, 'other': 5, 'mainly_numeric': 6, 'initialUpper': 4, 'contains_digit': 7, 'allUpper': 3, 'PADDING_TOKEN': 0}


In [11]:
print(type(trainSentences))

<class 'list'>


In [12]:
print(trainSentences[0])

[['Schartau', 'O'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'O'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'O'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [13]:
models.char2Idx={"PADDING_TOKEN":0, "<S>":1, "</S>":2, "<W>":3, "</W>":4}
for char in characters:
    models.char2Idx[char] = len(models.char2Idx)
models.char2Idx['UNKNOWN'] = len(models.char2Idx)
print(models.char2Idx)

{'8': 5, 'µ': 6, 'и': 261, 'к': 114, 't': 7, 'ý': 8, 'Å': 278, '別': 10, 'й': 12, 'PADDING_TOKEN': 0, 'z': 55, 'Т': 14, 'ε': 15, '東': 17, '公': 60, 'ό': 113, 'ą': 220, 'Ž': 18, '/': 328, '\x96': 19, 'т': 20, 'ν': 21, 'ά': 150, 'J': 22, 'н': 284, 'τ': 300, 'ı': 26, '\x9a': 32, 'σ': 29, '»': 30, 'F': 31, '</S>': 2, 'ø': 35, 'Ш': 214, 'ǒ': 37, ']': 38, '학': 40, 'ć': 64, 'أ': 42, '·': 9, '>': 43, '冲': 184, '_': 311, '章': 45, 'Ş': 46, 'ς': 176, 'б': 48, 'T': 289, '$': 49, '°': 50, 'ł': 159, '算': 51, 'γ': 52, '§': 233, 'Ö': 53, 'г': 54, '[': 59, 'b': 56, 'q': 57, 'Π': 13, '術': 286, '×': 61, 'ö': 62, 'C': 63, '½': 189, '李': 65, '\x94': 66, '³': 230, 'ó': 123, 'À': 67, 'ě': 275, 'з': 69, '별': 258, '¤': 82, 'Ä': 293, 'λ': 73, 'ب': 75, 'ă': 74, 'Ġ': 25, 'K': 76, 'オ': 77, 'Â': 78, '€': 79, 'ض': 80, '柯': 84, 'Š': 16, '루': 248, '南': 85, 'p': 86, 'č': 240, 'ô': 87, 'š': 252, '@': 88, 'ῦ': 238, '造': 90, 'ā': 70, '守': 92, 'Л': 93, 'д': 94, 'ʻ': 95, 'ť': 98, 'є': 97, 'j': 99, '\x99': 103, 'u': 101, 'ю': 

In [14]:
print(trainSentences[0]) 

[['Schartau', 'O'], ['sagte', 'O'], ['dem', 'O'], ['"', 'O'], ['Tagesspiegel', 'O'], ['"', 'O'], ['vom', 'O'], ['Freitag', 'O'], [',', 'O'], ['Fischer', 'O'], ['sei', 'O'], ['"', 'O'], ['in', 'O'], ['einer', 'O'], ['Weise', 'O'], ['aufgetreten', 'O'], [',', 'O'], ['die', 'O'], ['alles', 'O'], ['andere', 'O'], ['als', 'O'], ['überzeugend', 'O'], ['war', 'O'], ['"', 'O'], ['.', 'O']]


In [15]:
models.ft = fastText.load_model("../../fastText/wiki.de.bin")

In [16]:
print(models.nb_embedding_dims)
print(len(trainSentences[0]))

300
25


In [17]:
models.idx2Label = {v: k for k, v in models.label2Idx.items()}
print(len(models.label2Idx))
print(len(models.idx2Label))

19
19


# Compute Model

In [ ]:
# import importlib
# importlib.reload(models)

In [19]:
tmp_model_filename = 'model_lstm_germeval_2nd-level.h5'
# checkpoint = ModelCheckpoint(tmp_model_filename, verbose=1, save_best_only = True, monitor = 'val_acc')
history = utils.F1History(tmp_model_filename, devSet = devSentences)
model = models.get_model_lstm_v2()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         (None, None, 52)     0                                            
__________________________________________________________________________________________________
casing_input (InputLayer)       (None, None)         0                                            
__________________________________________________________________________________________________
char_embedding (TimeDistributed (None, None, 52, 32) 10688       char_input[0][0]                 
__________________________________________________________________________________________________
words_input (InputLayer)        (None, None, 300)    0                                            
__________________________________________________________________________________________________
case_embed

In [ ]:
#import importlib
#importlib.reload(utils)
# print(models.max_sequence_length)

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences, shuffle_data=True, batch_size=16), 
    validation_data = utils.NerSequence(devSentences, batch_size=256), 
    epochs = 10, callbacks = [history]
)

Epoch 1/10
1499/1500 [============================>.] - ETA: 0s - loss: 0.0071 - acc: 0.9969
New maximum F1 score: 0.4797843665768194 (before: 0) Saving to model_lstm_germeval_2nd-level.h5
1500/1500 [==============================] - 402s 268ms/step - loss: 0.0071 - acc: 0.9969 - val_loss: -0.0156 - val_acc: 0.9983
Epoch 2/10
1500/1500 [==============================] - 403s 269ms/step - loss: -0.0374 - acc: 0.9983 - val_loss: -0.0615 - val_acc: 0.9984
Epoch 3/10
1499/1500 [============================>.] - ETA: 0s - loss: -0.0866 - acc: 0.9984
New maximum F1 score: 0.5507246376811593 (before: 0.4797843665768194) Saving to model_lstm_germeval_2nd-level.h5
1500/1500 [==============================] - 397s 265ms/step - loss: -0.0867 - acc: 0.9984 - val_loss: -0.1126 - val_acc: 0.9984
Epoch 4/10
1500/1500 [==============================] - 398s 265ms/step - loss: -0.1396 - acc: 0.9985 - val_loss: -0.1647 - val_acc: 0.9980
Epoch 5/10
1500/1500 [==============================] - 398s 266ms/

In [ ]:
print(history.acc)
print(history.f1_scores)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
model.fit_generator(
    utils.NerSequence(trainSentences, shuffle_data=True, batch_size=2048), 
    validation_data = utils.NerSequence(devSentences, batch_size=256), 
    epochs = 10, callbacks = [history]
)

In [ ]:
model.load_weights(tmp_model_filename)

In [ ]:
true_labels, pred_labels = utils.predict_sequences(model, testSentences[:1000])
print(compute_f1(pred_labels, true_labels, models.idx2Label))

In [ ]:
true_labels, pred_labels = utils.predict_sequences(model, testSentences)
print(compute_f1(pred_labels, true_labels, models.idx2Label))

# Save final model

In [ ]:
import shutil, json
# copy file for best run
shutil.copyfile(tmp_model_filename, '../models/final_model_germeval_inner.h5')
shutil.copyfile(tmp_model_filename + '.indexes', '../models/final_model_germeval_inner.indexes')

In [ ]:
from keras_contrib.layers import CRF
def create_custom_objects():
    instanceHolder = {"instance": None}
    class ClassWrapper(CRF):
        def __init__(self, *args, **kwargs):
            instanceHolder["instance"] = self
            super(ClassWrapper, self).__init__(*args, **kwargs)
    def loss(*args):
        method = getattr(instanceHolder["instance"], "loss_function")
        return method(*args)
    def accuracy(*args):
        method = getattr(instanceHolder["instance"], "accuracy")
        return method(*args)
    return {"ClassWrapper": ClassWrapper ,"CRF": ClassWrapper, "loss": loss, "accuracy":accuracy}

finalmodel = load_model('../models/final_model_germeval_outer.h5', custom_objects=create_custom_objects())
true_labels, pred_labels = utils.predict_sequences(finalmodel, testSentences)
print(compute_f1(pred_labels, true_labels, models.idx2Label))

In [ ]:
f = open('germeval_output.tsv', 'w', encoding='UTF-8')
for i_sent, sent in enumerate(testSentences):
    for i_tok, tok in enumerate(sent):
        if tok[0] == 'PADDING_TOKEN':
            break
        correctlabel = models.idx2Label[true_labels[i_sent][i_tok]]
        guessedlabel = models.idx2Label[pred_labels[i_sent][i_tok]]
        line = "\t".join([str(i_tok+1), tok[0], correctlabel, correctlabel, guessedlabel, guessedlabel])
        f.write(line + '\n')
    f.write('\n')
f.close